In [30]:
# import dependencies
import pandas as pd
import json
import requests
from googleapiclient.discovery import build
from config import api_key

In [31]:
# Enter video id
# Snarky Puppy - Trinity
video_id = "CSiLPkUopBg"

In [32]:
def video_comments(video_id):
    # empty list for storing reply
    replies = []
  
    # creating youtube resource object
    youtube = build('youtube', 'v3', developerKey=api_key)
  
    # retrieve youtube video results
    video_response = youtube.commentThreads().list(part='snippet,replies', videoId=video_id).execute()
  
    # iterate video response
    while video_response:
        
        # extracting required info from each result object 
        for item in video_response['items']:
            
            # Extracting comments
            comment = item['snippet']['topLevelComment']['snippet']['textOriginal']
              
            # counting number of reply of comment
            replycount = item['snippet']['totalReplyCount']
  
            # if reply is there
            if replycount>0:
                
                # iterate through all reply
                for reply in item['replies']['comments']:
                    
                    # Extract reply
                    reply = reply['snippet']['textOriginal']
                      
                    # Store reply in list
                    replies.append(reply)
  
            # print comment with list of reply to text file
            with open('comments.txt', 'a', encoding="utf-8") as f:
                f.write(comment + str(replies) + '\n\n')

            print(comment, replies, end = '\n\n')
  
            # empty reply list
            replies = []
  
        # Again repeat on next page
        if 'nextPageToken' in video_response:
            video_response = youtube.commentThreads().list(
            part='snippet,replies',
            videoId=video_id,
            pageToken=video_response['nextPageToken']
            ).execute()
        else:
            break


In [ ]:
video_comments(video_id)

In [70]:
def video_comments_to_df(video_id):
    # creating youtube resource object
    youtube = build('youtube', 'v3', developerKey=api_key)
    
    # retrieve youtube video results
    video_response = youtube.commentThreads().list(part='snippet,replies', videoId=video_id).execute()

    # create empty dataframe
    df = pd.DataFrame(columns=['id', 'author', 'thumbsup', 'text', '# of replies', 'replies'])

    # iterate video response
    while video_response:

    # export comments to a dataframe
        for i in range(len(video_response['items'])):
            replies = []
            id = video_response['items'][i]['snippet']['topLevelComment']['id']
            author = video_response['items'][i]['snippet']['topLevelComment']['snippet']['authorDisplayName']
            text = video_response['items'][i]['snippet']['topLevelComment']['snippet']['textOriginal']
            replycount = video_response['items'][i]['snippet']['totalReplyCount']
            if replycount>0:
                for reply in video_response['items'][i]['replies']['comments']:
                    reply = reply['snippet']['textOriginal']
                    replies.append(reply)
            df = df.append({
                'id': id, 
                'author': author, 
                'thumbsup': int(video_response['items'][i]['snippet']['topLevelComment']['snippet']['likeCount']),
                'text': text, 
                '# of replies': replycount, 
                'replies': replies 
                }, 
                ignore_index=True)
        if 'nextPageToken' in video_response:
            video_response = youtube.commentThreads().list(
            part='snippet,replies',
            videoId=video_id,
            pageToken=video_response['nextPageToken']
            ).execute()
        else:
            break

    # change data types
    df['thumbsup'] = df['thumbsup'].astype(int)
    df['# of replies'] = df['# of replies'].astype(int)

    return df

In [71]:
video_comments_to_df(video_id)

,id,author,thumbsup,text,# of replies,replies
0,UgyAVMOl8OQUi-RoBh54AaABAg,zoomlensshow,0,Crazy how well the two drum sets play together...,0,[]
1,Ugwmer_nvfZvQZYymTN4AaABAg,Rudy Pry,0,I fell like God probably has Snarky Puppy on h...,0,[]
2,UgzM1G7dCoom3YKTnel4AaABAg,Mladen Zivkovic,0,https://www.youtube.com/shorts/8noX0Fu36fI,0,[]
3,UgwX15tneuS0JK_H0Ch4AaABAg,TheProf82,0,Are all 16 tracks visually registered like thi...,1,[They're releasing them slowly one by one]
4,Ugw_3QlxWv_vWnZbEOt4AaABAg,steamer482,0,Is that Jack Conte I see at the 8:22 and 10:48...,0,[]
...,...,...,...,...,...,...
1296,Ugx0hC0V5MUK3nuYo6x4AaABAg,Jordan Allen,54,Back to the roots of live album recording! Tod...,0,[]
1297,Ugxavri1ffOFqjbWGfN4AaABAg,7 F,0,nice,0,[]
1298,UgwjqV3A9LvLoF8ddK54AaABAg,BrandonSorianoMusic,4,Was not expecting it to go all Hans Zimmer on ...,0,[]
1299,UgxHzEujoGsZ02-WCXV4AaABAg,Devaughn Goodlow,1,Watching y’all at the show in kc y’all killing...,0,[]


In [73]:
# export to csv
df.to_csv('comments.csv', index=False)


In [68]:
# total reply checker

# for "Trinity", total replies should be 1868
total_replies = len(df) + sum(df['# of replies'])
print(f'Total replies: {total_replies}')

# SUCCESS


Total replies: 1868
